# Meeting Notes to Google Docs Converter

This notebook converts markdown-formatted meeting notes to a well-formatted Google Doc using the Google Docs API.

In [ ]:
# Install required packages
!pip install -q google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import re
import json
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os
import pickle

## Define the Meeting Notes

In [ ]:
# Store the markdown meeting notes as a string variable
meeting_notes = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes"""

## Google Docs API Authentication

In [ ]:
def authenticate_google_docs():
    """Authenticate with Google Docs API using OAuth2"""
    SCOPES = ['https://www.googleapis.com/auth/documents']
    creds = None

    try:
        # In Colab, we'll use the built-in authentication
        from google.colab import auth
        auth.authenticate_user()
        
        # For Colab, we'll use application default credentials
        from google.auth import default
        creds, project = default(scopes=SCOPES)
        
        print('Authentication successful!')
        return build('docs', 'v1', credentials=creds)
        
    except Exception as e:
        print(f'Error during authentication: {str(e)}')
        import traceback
        traceback.print_exc()
        raise

## Parse Markdown and Format Google Doc Content

In [ ]:
def parse_markdown_to_doc_format(markdown_text):
    """Parse markdown text and convert it to Google Docs format"""
    lines = markdown_text.strip().split('\n')
    doc_elements = []
    
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        # Process different markdown elements
        if line.startswith('# '):  # H1
            # Create paragraph with H1 style
            doc_elements.append({
                'paragraph': {
                    'elements': [{
                        'textRun': {
                            'content': line[2:] + '\n'  # Remove '# ' and add newline
                        }
                    }],
                    'paragraphStyle': {
                        'namedStyleType': 'HEADING_1'
                    }
                }
            })
        
        elif line.startswith('## '):  # H2
            # Create paragraph with H2 style
            doc_elements.append({
                'paragraph': {
                    'elements': [{
                        'textRun': {
                            'content': line[3:] + '\n'  # Remove '## ' and add newline
                        }
                    }],
                    'paragraphStyle': {
                        'namedStyleType': 'HEADING_2'
                    }
                }
            })
        
        elif line.startswith('### '):  # H3
            # Create paragraph with H3 style
            doc_elements.append({
                'paragraph': {
                    'elements': [{
                        'textRun': {
                            'content': line[4:] + '\n'  # Remove '### ' and add newline
                        }
                    }],
                    'paragraphStyle': {
                        'namedStyleType': 'HEADING_3'
                    }
                }
            })
        
        elif line.startswith('- [ ]'):  # Unchecked checkbox
            # Extract content after checkbox
            content = line[5:]  # Remove '- [ ]'
            # Process @mentions in the content
            elements = process_mentions(content)
            
            # Create a list item with checkbox
            doc_elements.append({
                'paragraph': {
                    'elements': [
                        {
                            'textRun': {
                                'content': '[ ] ',  # Simple checkbox representation
                            }
                        }
                    ] + elements,
                    'paragraphStyle': {
                        'namedStyleType': 'NORMAL_TEXT'
                    }
                }
            })
        
        elif line.startswith('- '):  # Regular bullet point
            # Calculate indentation level
            indent_level = 0
            original_line = line
            while original_line.startswith('  '):
                indent_level += 1
                original_line = original_line[2:]
            
            content = original_line[2:]  # Remove '- '
            elements = process_mentions(content)
            
            # Create the paragraph
            paragraph = {
                'paragraph': {
                    'elements': [
                        {
                            'textRun': {
                                'content': '* ' + elements[0]['textRun']['content']
                            }
                        }
                    ] + elements[1:],
                    'paragraphStyle': {
                        'namedStyleType': 'NORMAL_TEXT',
                        'indentFirstLine': {
                            'magnitude': indent_level,
                            'unit': 'EMU'
                        },
                        'leftIndent': {
                            'magnitude': indent_level * 36.0,
                            'unit': 'PT'
                        }
                    }
                }
            }
            
            doc_elements.append(paragraph)
        
        elif line.startswith('* '):  # Asterisk bullet point
            # Calculate indentation level
            indent_level = 0
            original_line = line
            while original_line.startswith('  '):
                indent_level += 1
                original_line = original_line[2:]
            
            content = original_line[2:]  # Remove '* '
            elements = process_mentions(content)
            
            # Create the paragraph
            paragraph = {
                'paragraph': {
                    'elements': [
                        {
                            'textRun': {
                                'content': 'o ' + elements[0]['textRun']['content']
                            }
                        }
                    ] + elements[1:],
                    'paragraphStyle': {
                        'namedStyleType': 'NORMAL_TEXT',
                        'indentFirstLine': {
                            'magnitude': indent_level,
                            'unit': 'EMU'
                        },
                        'leftIndent': {
                            'magnitude': indent_level * 36.0,
                            'unit': 'PT'
                        }
                    }
                }
            }
            
            doc_elements.append(paragraph)
        
        elif line.startswith('---'):  # Footer separator
            # Process next lines as footer until we reach the end or an empty line
            i += 1
            while i < len(lines) and lines[i].strip():
                footer_line = lines[i].strip()
                doc_elements.append({
                    'paragraph': {
                        'elements': [{
                            'textRun': {
                                'content': footer_line + '\n',
                            }
                        }],
                        'paragraphStyle': {
                            'namedStyleType': 'NORMAL_TEXT'
                        }
                    }
                })
                i += 1
            continue  # Skip incrementing i at the end of the loop
        
        elif line:  # Regular text
            # Process regular text that's not a special element
            elements = process_mentions(line)
            
            doc_elements.append({
                'paragraph': {
                    'elements': elements,
                    'paragraphStyle': {
                        'namedStyleType': 'NORMAL_TEXT'
                    }
                }
            })
        
        i += 1
    
    return doc_elements

def process_mentions(text):
    """Process @mentions in text and apply distinct styling"""
    # Find all @mentions
    mention_pattern = re.compile(r'(@\w+)')
    parts = mention_pattern.split(text)
    
    elements = []
    for part in parts:
        if mention_pattern.match(part):
            # This is a mention - apply distinct styling
            elements.append({
                'textRun': {
                    'content': part + ' ',
                    'textStyle': {
                        'bold': True,
                        'foregroundColor': {
                            'color': {
                                'rgbColor': {
                                    'blue': 1.0,
                                    'green': 0.0,
                                    'red': 0.0
                                }
                            }
                        }
                    }
                }
            })
        else:
            # Regular text
            elements.append({
                'textRun': {
                    'content': part,
                    'textStyle': {}
                }
            })
    
    # Add newline at the end
    if elements:
        elements[-1]['textRun']['content'] += '\n'
    
    return elements

## Create Google Doc

In [ ]:
def create_google_doc(docs_service, title, content_elements):
    """Create a new Google Doc with the specified content"""
    try:
        # Create a new document
        document = docs_service.documents().create(body={'title': title}, 
                                                   fields='documentId').execute()
        document_id = document.get('documentId')
        print(f'Created document with ID: {document_id}')
        
        # Build the requests to populate the document
        requests = []
        
        # Insert each element
        insert_index = 1  # Start inserting after the first character (which is always a placeholder)
        for element in content_elements:
            # First, insert the text content
            text_content = ''.join([e['textRun']['content'] for e in element['paragraph']['elements']])
            
            requests.append({
                'insertText': {
                    'location': {
                        'index': insert_index
                    },
                    'text': text_content
                }
            })
            
            # Then apply paragraph style
            requests.append({
                'updateParagraphStyle': {
                    'range': {
                        'startIndex': insert_index,
                        'endIndex': insert_index + len(text_content)
                    },
                    'paragraphStyle': element['paragraph']['paragraphStyle'],
                    'fields': 'namedStyleType,indentFirstLine,leftIndent'
                }
            })
            
            # Apply text styles for each element within the paragraph
            char_index = insert_index
            for elem in element['paragraph']['elements']:
                if 'textStyle' in elem['textRun'] and elem['textRun']['textStyle']:
                    text_len = len(elem['textRun']['content'])
                    if text_len > 0:  # Only apply style if there's content
                        style_request = {
                            'updateTextStyle': {
                                'range': {
                                    'startIndex': char_index,
                                    'endIndex': char_index + text_len
                                },
                                'textStyle': elem['textRun']['textStyle'],
                                'fields': 'bold,foregroundColor'
                            }
                        }
                        requests.append(style_request)
                char_index += len(elem['textRun']['content'])
            
            insert_index += len(text_content)
        
        # Execute the batch update
        if requests:
            result = docs_service.documents().batchUpdate(
                documentId=document_id, body={'requests': requests}).execute()
        
        print(f'Document successfully created! Access it at: https://docs.google.com/document/d/{document_id}/edit')
        return document_id
        
    except Exception as error:
        print(f'An error occurred: {error}')
        import traceback
        traceback.print_exc()
        return None

## Main Execution Function

In [ ]:
def main():
    """Main function to convert markdown to Google Doc"""
    try:
        # Authenticate with Google Docs API
        print('Authenticating with Google Docs API...')
        docs_service = authenticate_google_docs()
        print('Authentication successful!')
        
        # Parse the markdown to document format
        print('Parsing markdown content...')
        doc_elements = parse_markdown_to_doc_format(meeting_notes)
        print(f'Parsing complete! Processed {len(doc_elements)} document elements.')
        
        # Create the Google Doc
        print('Creating Google Doc...')
        doc_id = create_google_doc(docs_service, 'Product Team Sync - May 15, 2023', doc_elements)
        
        if doc_id:
            doc_url = f'https://docs.google.com/document/d/{doc_id}/edit'
            print(f'Successfully created document: {doc_url}')
            return doc_url
        else:
            print('Failed to create document')
            return None
            
    except Exception as e:
        print(f'Error during execution: {str(e)}')
        import traceback
        traceback.print_exc()
        return None

## Run the Converter

In [ ]:
if __name__ == '__main__':
    main()